In [2]:
import os
import unittest
import abc

from typing import Any
from source.core.dataclasses import field, dataclass
from source.core.confg import create_config_class, resolve_defaults
from source.core.registry import RegistryMeta
from source.core.discriminated_union import DiscriminatedUnion

In [31]:
import os
import unittest
import abc

from typing import Any
from source.core.dataclasses import field, dataclass
from source.core.confg import create_config_class, resolve_defaults
from source.core.registry import RegistryMeta
from source.core.discriminated_union import DiscriminatedUnion


class AB:
    @resolve_defaults
    def __init__(
        self, a: int = 1, b: int = field(default_factory=lambda: 2)
    ) -> None:
        self.a = a
        self.b = b

    def __call__(self) -> int:
        return self.a + self.b

@create_config_class(AB.__init__)
class ABConfigClass:
    pass

class FooRegistry(metaclass=RegistryMeta):
    @abc.abstractmethod
    def foo(self) -> int:
        pass


@dataclass
class Foo(FooRegistry):
    ab_cfg: ABConfigClass = field(default_factory=ABConfigClass)

    def foo(self):
        ab = AB(**self.ab_cfg.asdict())
        return ab()

@dataclass
class Bar(FooRegistry):
    def foo(self) -> int:
        return 10


@FooRegistry.register()
class FooRoster(DiscriminatedUnion):
    pass


In [3]:
import os
import unittest
import abc

from typing import Any
from source.core.dataclasses import field, dataclass
from source.core.confg import create_config_class, resolve_defaults
from source.core.registry import RegistryMeta
from source.core.discriminated_union import DiscriminatedUnion


@dataclass
class MM:
    def __post_init_post_parse__(self):
        pass
# @dataclass
# class FM(ModelRegistry):
#     a: int = 1

@dataclass
class s2s_base(MM):
    def __post_init_post_parse__(self):
        super().__post_init_post_parse__()

@dataclass
class s2s(s2s_base, ModelRegistry):
    a: int
    def __post_init_post_parse__(self):
        super().__post_init_post_parse__()

@ModelRegistry.register()
class ModelRoster(DiscriminatedUnion):
    pass

In [4]:
ModelRoster.__pydantic_model__.schema_json()

'{"title": "ModelRoster", "type": "object", "properties": {"s2s": {"$ref": "#/definitions/s2s"}}, "definitions": {"s2s": {"title": "s2s", "type": "object", "properties": {"a": {"title": "A", "type": "integer"}}, "required": ["a"]}}}'

In [6]:
ModelRoster(s2s=s2s(12)).value

s2s(a=12)

In [34]:
ABConfigClass().__repr__()

'ABConfigClass(a=1, b=2)'

In [23]:
foo = Foo()

In [24]:
foo.REGISTRY

{'Foo': __main__.Foo, 'Bar': __main__.Bar}

In [30]:
FooRoster()

TypeError: 'type' object is not subscriptable

In [11]:
ab()

6

In [2]:
def test_func(a:int, b:bool, c: float = 3.5):
    pass

In [4]:
def get_config_class():
    @create_config_class(test_func)
    class ConfigClass:
        pass
    return ConfigClass

In [5]:
cfg_class = get_config_class()

In [14]:
cfg_class.a

AttributeError: type object 'ConfigClass' has no attribute 'a'

In [7]:
cfg_class.__pydantic_model__.schema_json()

'{"title": "ConfigClass", "type": "object", "properties": {"a": {"title": "A", "type": "integer"}, "b": {"title": "B", "type": "boolean"}, "c": {"title": "C", "default": 3.5, "type": "number"}}, "required": ["a", "b"]}'